In [50]:
import requests, json
import pandas as pd

f = open('config.json','r')
key = json.load(f)
futapikeytest = key['futapikeytest']
futapikeylive = key['futapikeylive']
campeonato_id = 10
urlroot = "https://api.api-futebol.com.br"
head = {'Authorization': 'Bearer ' + futapikeylive}
data = {'app' : 'aaaaa'}

dictpart1 = {
    "Id_Partida": [],
    "Rodada": [],
    "Placar": [],
    "Time_Mandante": [],
    "Time_Visitante": [],
    "Gols_Mandante": [],
    "Gols_Visitante": [],
    "Data_Realizacao": [],
    "Hora_Realizacao": [],
    "Estadio": [],
    "Posse_de_bola_mandante": [], 
    "Finalizacao_mandante": [],
    "Dribles_mandante": [],
    "Posse_de_bola_visitante": [],
    "Finalizacao_visitante": [],
    "Dribles_visitante": [],
}

for rodada in range(1,9):

    url = urlroot + f'/v1/campeonatos/{campeonato_id}/rodadas/{rodada}'
    response = requests.post(url, json=data, headers=head)
    rodada = response.json()

    for partida in rodada['partidas']:
        urlpartida = urlroot + f"/v1/partidas/{partida['partida_id']}"
        response = requests.post(urlpartida, json=data, headers=head)
        resposta = response.json()
        
        dictpart1['Id_Partida'].append(resposta['partida_id']) if resposta['partida_id'] else dictpart1['Id_Partida'].append("")
        dictpart1['Rodada'].append(resposta['rodada'])
        dictpart1['Placar'].append(resposta['placar'])
        dictpart1['Time_Mandante'].append(resposta['time_mandante']['nome_popular'])
        dictpart1['Time_Visitante'].append(resposta['time_visitante']['nome_popular'])
        dictpart1['Gols_Mandante'].append(resposta['placar_mandante'])
        dictpart1['Gols_Visitante'].append(resposta['placar_visitante'])
        dictpart1['Data_Realizacao'].append(resposta['data_realizacao'])
        dictpart1['Hora_Realizacao'].append(resposta['hora_realizacao'])
        dictpart1['Estadio'].append(resposta['estadio']["nome_popular"])
        
        if isinstance(resposta["estatisticas"]["mandante"], (dict)):
            dictpart1['Posse_de_bola_mandante'].append(resposta["estatisticas"]["mandante"]["posse_de_bola"])
            dictpart1['Finalizacao_mandante'].append(resposta["estatisticas"]["mandante"]['finalizacao']['total'])
            dictpart1['Dribles_mandante'].append(resposta["estatisticas"]["mandante"]['dribes_bem_sucedidos'])
        else:
            dictpart1['Posse_de_bola_mandante'].append("NaN")
            dictpart1['Finalizacao_mandante'].append("NaN")
            dictpart1['Dribles_mandante'].append("NaN")
       
        if isinstance(resposta["estatisticas"]["visitante"], (dict)): 
            dictpart1['Posse_de_bola_visitante'].append(resposta["estatisticas"]["visitante"]["posse_de_bola"])
            dictpart1['Finalizacao_visitante'].append(resposta["estatisticas"]["visitante"]['finalizacao']['total'])
            dictpart1['Dribles_visitante'].append(resposta["estatisticas"]["visitante"]["posse_de_bola"])
        else:
            dictpart1['Posse_de_bola_visitante'].append("NaN")
            dictpart1['Finalizacao_visitante'].append("NaN")
            dictpart1['Dribles_visitante'].append("NaN")

df = pd.DataFrame(dictpart1)

KeyError: 'partida_id'

In [52]:
display(df)

,Id_Partida,Rodada,Placar,Time_Mandante,Time_Visitante,Gols_Mandante,Gols_Visitante,Data_Realizacao,Hora_Realizacao,Estadio,Posse_de_bola_mandante,Finalizacao_mandante,Dribles_mandante,Posse_de_bola_visitante,Finalizacao_visitante,Dribles_visitante
0,2167,3ª Rodada,Palmeiras 1x1 Corinthians,Palmeiras,Corinthians,1,1,12/06/2021,19:00,Allianz Parque,51%,12,7,48%,9,48%
1,2168,3ª Rodada,Santos 0x0 Juventude,Santos,Juventude,0,0,12/06/2021,19:00,Vila Belmiro,75%,17,12,24%,2,24%
2,2166,3ª Rodada,Flamengo 2x0 América-MG,Flamengo,América-MG,2,0,13/06/2021,16:00,Maracanã,64%,20,17,35%,9,35%
3,2169,3ª Rodada,Atlético-MG 1x0 São Paulo,Atlético-MG,São Paulo,1,0,13/06/2021,16:00,Mineirão,42%,9,6,57%,5,57%
4,2170,3ª Rodada,Grêmio 0x1 Athletico-PR,Grêmio,Athletico-PR,0,1,13/06/2021,16:00,Arena do Grêmio,63%,10,8,36%,9,36%
5,2171,3ª Rodada,Bahia 0x1 Internacional,Bahia,Internacional,0,1,13/06/2021,20:30,Pituaçu,67%,14,3,32%,6,32%
6,2172,3ª Rodada,Fortaleza 1x0 Sport,Fortaleza,Sport,1,0,13/06/2021,20:30,Castelão (CE),58%,13,7,41%,5,41%
7,2173,3ª Rodada,Bragantino 2x2 Fluminense,Bragantino,Fluminense,2,2,13/06/2021,20:30,Nabi Abi Chedid,47%,16,9,52%,15,52%
8,2174,3ª Rodada,Chapecoense 0x0 Ceará,Chapecoense,Ceará,0,0,13/06/2021,20:30,Arena Condá,38%,8,3,61%,14,61%
9,2175,3ª Rodada,Cuiabá 2x1 Atlético-GO,Cuiabá,Atlético-GO,2,1,21/07/2021,19:00,Arena Pantanal,55%,20,5,44%,9,44%
